In [1]:
from pypdf import PdfReader
import itertools
import numpy as np

In [ ]:
folder = "documents"
ukr_texts = ["Конституція.pdf", "Енеїда.pdf", "Кучма.pdf", "Хто.pdf"]

large_text = ""

for text in ukr_texts:
    reader = PdfReader(f"{folder}/{text}")
    for page in reader.pages:
        large_text += page.extract_text() + "\n"


In [ ]:
alph = list('АаБбВвГгҐґДдЕеЄєЖжЗзИиІіЇїЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЬьЮюЯя')
alph_low = list(' абвгґдеєжзиіїйклмнопрстуфхцчшщьюя')

lang = []

for c in large_text:
    if c in alph:
        lang.append(str.lower(c))
    elif lang[-1] != ' ':
        lang.append(' ')

alph = [' '] + alph

RING = {}
for char in alph_low:
    pos = alph_low.index(char)
    RING[char] = pos

In [ ]:
lang_1_counts = {k: lang.count(k) for k in alph_low}

bigrams = np.transpose([lang[:-1], lang[1:]]).tolist()

lang_2_counts = {(c1, c2): bigrams.count([c1, c2]) for c1, c2 in itertools.product(alph_low, alph_low)}

print(lang_1_counts)
print(lang_2_counts)


In [ ]:
def index_of_coincidence(l_gram_counts):
    counts = np.array(list(l_gram_counts.values()))
    N = counts.sum()
    I = sum(count * (count - 1) for count in counts) / (N * (N - 1))

    return I

def avg_entropy(l_gram_counts):
    norm = len(l_gram_counts.keys()[0])
    counts = np.array(list(l_gram_counts.values()))
    probs = counts / counts.sum()
    H = - sum(p * np.log(p) if p != 0 else 0 for p in probs) / norm

    return H


avg_entropy(lang_1_counts)


In [ ]:
def vigenere_encrypt(text: str, key: str):
    res = ""
    r = len(key)
    for i in range(0, len(text), r):
        for j in range(r):
            if i + j == len(text):
                break
            id = (RING[text[i + j]] + RING[key[j]]) % len(alph_low)
            res += alph_low[id]

    return res

In [ ]:
def affine_ecrypt(text: str, a: int, b: int):
    if np.gcd(a, len(alph_low)) != 1:
        raise RuntimeError("Incorrect key, gcd(a, n) != 1")
    
    res = ""
    for i in range(0, len(text)):
        id = (a * RING[text[i]] + b) % len(alph_low)
        res += alph_low[id]

    return res

def affine_bigram_ecrypt():